# Create Hospital Metric / Patient Survey Data Frame

### First we read our hospital survey data files into one dataframe and assign the filenames containing hospital IDs to a new column in the dataframe. Files were first downloaded from:
http://www.healthcarereportcard.illinois.gov/

In [1]:
#combine data files for each hospital in the directory

import pandas as pd
import glob, os

files = glob.glob('/home/tjd/InsightFiles/IllinoisReportCards/hospital*') #grabs all files from the directory (n = 184)

#concatenate files but also add file name as a new column to the df

survey_df = pd.concat([pd.read_csv(fp).assign(New=os.path.basename(fp).split('.')[0]) for fp in files])

#rename the new column

survey_df.rename(columns={'New':'Hosp_ID'}, inplace=True)
survey_df.head(5)

,measure_set_name,measure_name,hospital_name,value,date_start,date_end,Hosp_ID
0,CMS 30-day Mortality,Pneumonia 30-Day Mortality Rate,Kirby Medical Center,13.8%,07/01/2015,06/30/2018,hospital_101294
1,CMS 30-day Mortality,Heart Failure 30-Day Mortality Rate,Kirby Medical Center,NaN,07/01/2015,06/30/2018,hospital_101294
2,CMS 30-day Mortality,Heart Attack 30-Day Mortality Rate,Kirby Medical Center,NaN,07/01/2015,06/30/2018,hospital_101294
3,Inpatient Mortality,Risk-adjusted Mortality Rate: Inpatient Bypass...,Kirby Medical Center,NaN,NaN,NaN,hospital_101294
4,Inpatient Mortality,Risk-adjusted Mortality Rate: Heart Attack Dea...,Kirby Medical Center,too few cases,01/01/2015,12/31/2015,hospital_101294


In [2]:
#get number and list of hospital names to use when scraping Glassdoor reviews

numHosp = len(survey_df.hospital_name.unique())
print(numHosp)

names = pd.DataFrame(survey_df.hospital_name.unique())
names.to_csv("HospNamesList.csv")

##a few hospitals have the same exact name, but different ID, hence 181 != 184 files

181


In [3]:
#create unique IDs for each hospital to deal with duplicate hospital names
#pivot data to get one row per hospital 

survey_df['Hospital'] = survey_df['Hosp_ID'].str.cat(survey_df['hospital_name'], sep = ': ')
survey_df = survey_df.drop_duplicates(['Hosp_ID', 'measure_name', 'hospital_name'])
survey_df_wide = pd.pivot(survey_df, index = 'Hospital', columns = 'measure_name', values = 'value').reset_index()
survey_df_wide.shape #184, 281

survey_df_wide.head(5)

measure_name,Hospital,Accidental Puncture and Laceration,"Accidental Puncture and Laceration, Pediatric",Acute Mental Illness Beds\r\n,Admit decision time to ED departure time for admitted patients,Bi-Lateral Cardiac Catheterization,Birth Trauma,Blood Transfusion Reaction,Bypass Hours,Central Line Associated Bloodstream Infections in Pediatric Medical ICU,...,Total RN Nursing Hours per Patient Day (Critical Care),Total RN Nursing Hours per Patient Day (Medical-Surgical),Total RN Nursing Hours per Patient Day (Mother/Baby),Total RN Nursing Hours per Patient Day (NICU),Total Vaginal Births After Cesarean,Uncomplicated Vaginal Births After Cesarean,Unexpected Deaths,Volume: CT Scan Head (Inpatient),Volume: CT Scan Head (Outpatient),Wound Complications in Abdominal Wall Surgery
0,hospital_101152: Graham Hospital,Statistically significantly better or performi...,Statistically significantly better or performi...,0,79 minutes,too few cases,too few cases,NaN,NaN,NaN,...,16.93 hours,4.76 hours,18.01 hours,0 hours,108.7per 1000,too few cases,Statistically significantly better or performi...,311,"1,673",Statistically significantly better or performi...
1,hospital_101153: Alton Memorial Hospital,Not statistically significantly better or worse,Statistically significantly better or performi...,20,55 minutes,5.43per 1000,too few cases,NaN,NaN,NaN,...,15.11 hours,6.9 hours,8.45 hours,0 hours,too few cases,too few cases,Statistically significantly better or performi...,"1,649","3,689",Statistically significantly better or performi...
2,hospital_101154: Amita Health Saint Joseph Med...,Not statistically significantly better or worse,Statistically significantly worse,31,130 minutes,26.32per 1000,too few cases,NaN,2.34 hours,NaN,...,16.73 hours,6.81 hours,12.17 hours,0 hours,57.97per 1000,65.93per 1000,Not statistically significantly better or worse,"4,640","7,300",Statistically significantly better or performi...
3,hospital_101155: Loyola Gottlieb Memorial Hosp...,Statistically significantly better or performi...,Statistically significantly better or performi...,12,140 minutes,4.33per 1000,NaN,NaN,71.11 hours,NaN,...,15.28 hours,7.12 hours,0 hours,0 hours,NaN,NaN,Statistically significantly worse,"1,268","3,250",Statistically significantly better or performi...
4,hospital_101156: Northshore Univ HS Evanston H...,Not statistically significantly better or worse,Statistically significantly better or performi...,21,164 minutes,3.68per 1000,Not statistically significantly better or worse,NaN,NaN,NaN,...,16.26 hours,5.75 hours,8.43 hours,11.84 hours,344.58per 1000,361.17per 1000,Statistically significantly better or performi...,"4,479","4,942",Not statistically significantly better or worse


In [4]:
#pull out relevant columns that may inform nurse turnover
#there is a lot more information than we need to build a baseline model; more features could be added in later iterations

survey_df_wide_rel = pd.DataFrame(survey_df_wide[['Hospital', 'Doctors Always Communicated Well','Emergency Department Visits, Total',
 'Number of Beds','Nurses Always Communicated Well',
 'Patients Always Received Help As Soon As They Wanted',
 'Patients Would Definitely Recommend This Hospital to Friends and Family',
 'Percent RN Nursing Staff Hours (Critical Care)',
 'Percentage of Nursing Hours Worked by Hospital Employed RNs (Critical Care)',
 'RN Turnover Rate in the Critical Care Unit',
 'RN Turnover Rate in the Medical-Surgical Unit',
 'RN Turnover Rate in the Mother/Baby Unit',
 'Time from ED arrival to diagnostic evaluation by a qualified medical professional','Percent RN Nursing Staff Hours (Critical Care)','Total RN Nursing Hours per Patient Day (Critical Care)',
 'Total RN Nursing Hours per Patient Day (Medical-Surgical)', 'Percentage of Nursing Hours Worked by Hospital Employed RNs (Medical-Surgical)','Heart Attack 30-Day Mortality Rate','Pneumonia 30-Day Mortality Rate', 'Percent RN Nursing Staff Hours (Medical-Surgical)','Median Length of Stay for DRG 638, Diabetes with Complications','Median Length of Stay: Septicemia with Multiple Complications']])

In [5]:
#initially we use 22 features of the 281 in the data set

survey_df_wide_rel.shape #184, 22
survey_df_wide_rel.head()

measure_name,Hospital,Doctors Always Communicated Well,"Emergency Department Visits, Total",Number of Beds,Nurses Always Communicated Well,Patients Always Received Help As Soon As They Wanted,Patients Would Definitely Recommend This Hospital to Friends and Family,Percent RN Nursing Staff Hours (Critical Care),Percentage of Nursing Hours Worked by Hospital Employed RNs (Critical Care),RN Turnover Rate in the Critical Care Unit,...,Time from ED arrival to diagnostic evaluation by a qualified medical professional,Percent RN Nursing Staff Hours (Critical Care),Total RN Nursing Hours per Patient Day (Critical Care),Total RN Nursing Hours per Patient Day (Medical-Surgical),Percentage of Nursing Hours Worked by Hospital Employed RNs (Medical-Surgical),Heart Attack 30-Day Mortality Rate,Pneumonia 30-Day Mortality Rate,Percent RN Nursing Staff Hours (Medical-Surgical),"Median Length of Stay for DRG 638, Diabetes with Complications",Median Length of Stay: Septicemia with Multiple Complications
0,hospital_101152: Graham Hospital,85%,"13,205 patients",87,80%,68%,60%,93.18%,89.31%,0 %,...,9 minutes,93.18%,16.93 hours,4.76 hours,56.55%,NaN,18%,63.88 %,2 days,5 days
1,hospital_101153: Alton Memorial Hospital,76%,"35,566 patients",181,78%,64%,71%,90.26%,89.5%,7.41 %,...,16 minutes,90.26%,15.11 hours,6.9 hours,77.08%,12.6%,17.2%,92.03 %,3 days,4 days
2,hospital_101154: Amita Health Saint Joseph Med...,76%,"57,237 patients",498,77%,63%,63%,90.65%,93.52%,17.76 %,...,40 minutes,90.65%,16.73 hours,6.81 hours,71.54%,11.9%,16.6%,66.85 %,3 days,5 days
3,hospital_101155: Loyola Gottlieb Memorial Hosp...,78%,"25,358 patients",247,78%,66%,63%,95.07%,95.01%,3.33 %,...,29 minutes,95.07%,15.28 hours,7.12 hours,77.65%,12.4%,15.7%,79.32 %,2.5 days,5 days
4,hospital_101156: Northshore Univ HS Evanston H...,81%,"35,149 patients",354,80%,64%,77%,83.26%,83.84%,13.85 %,...,37 minutes,83.26%,16.26 hours,5.75 hours,59.36%,9.4%,12.8%,60.44 %,3 days,5 days


There are many extranneous characters in the dataframe that will cause problems when working with the dataframe, thus we need to strip them out (i.e., %, strings, commas).

In [6]:
#clean % symbols and unnecessary words from dataframe
#units should not be inside the dataframe with the data!

survey_df_wide_rel = survey_df_wide_rel.replace(regex=r'\%', value='')
survey_df_wide_rel = survey_df_wide_rel.replace(regex=r'minutes', value='')
survey_df_wide_rel = survey_df_wide_rel.replace(regex=r'hours', value='')
survey_df_wide_rel = survey_df_wide_rel.replace(regex=r'patients', value='')
survey_df_wide_rel = survey_df_wide_rel.replace(regex=r'\,', value='')
survey_df_wide_rel = survey_df_wide_rel.replace(regex=r'days', value='')
survey_df_wide_rel = survey_df_wide_rel.replace(regex=r'too few cases', value='nan')
survey_df_wide_rel.head()

measure_name,Hospital,Doctors Always Communicated Well,"Emergency Department Visits, Total",Number of Beds,Nurses Always Communicated Well,Patients Always Received Help As Soon As They Wanted,Patients Would Definitely Recommend This Hospital to Friends and Family,Percent RN Nursing Staff Hours (Critical Care),Percentage of Nursing Hours Worked by Hospital Employed RNs (Critical Care),RN Turnover Rate in the Critical Care Unit,...,Time from ED arrival to diagnostic evaluation by a qualified medical professional,Percent RN Nursing Staff Hours (Critical Care),Total RN Nursing Hours per Patient Day (Critical Care),Total RN Nursing Hours per Patient Day (Medical-Surgical),Percentage of Nursing Hours Worked by Hospital Employed RNs (Medical-Surgical),Heart Attack 30-Day Mortality Rate,Pneumonia 30-Day Mortality Rate,Percent RN Nursing Staff Hours (Medical-Surgical),"Median Length of Stay for DRG 638, Diabetes with Complications",Median Length of Stay: Septicemia with Multiple Complications
0,hospital_101152: Graham Hospital,85,13205,87,80,68,60,93.18,89.31,0,...,9,93.18,16.93,4.76,56.55,NaN,18,63.88,2,5
1,hospital_101153: Alton Memorial Hospital,76,35566,181,78,64,71,90.26,89.5,7.41,...,16,90.26,15.11,6.9,77.08,12.6,17.2,92.03,3,4
2,hospital_101154: Amita Health Saint Joseph Med...,76,57237,498,77,63,63,90.65,93.52,17.76,...,40,90.65,16.73,6.81,71.54,11.9,16.6,66.85,3,5
3,hospital_101155: Loyola Gottlieb Memorial Hosp...,78,25358,247,78,66,63,95.07,95.01,3.33,...,29,95.07,15.28,7.12,77.65,12.4,15.7,79.32,2.5,5
4,hospital_101156: Northshore Univ HS Evanston H...,81,35149,354,80,64,77,83.26,83.84,13.85,...,37,83.26,16.26,5.75,59.36,9.4,12.8,60.44,3,5


Now we investigate the missing values to determine what to do about them. 

In [7]:
#count the number of missing values to inform decision on how to deal with NaN

cols = survey_df_wide_rel.columns
survey_df_wide_rel.isna().sum()
#print(cols)

measure_name
Hospital                                                                              0
Doctors Always Communicated Well                                                      7
Emergency Department Visits, Total                                                    7
Number of Beds                                                                        0
Nurses Always Communicated Well                                                       7
Patients Always Received Help As Soon As They Wanted                                  7
Patients Would Definitely Recommend This Hospital to Friends and Family               7
Percent RN Nursing Staff Hours (Critical Care)                                        4
Percentage of Nursing Hours Worked by Hospital Employed RNs (Critical Care)           8
RN Turnover Rate in the Critical Care Unit                                            9
RN Turnover Rate in the Medical-Surgical Unit                                         9
RN Turnover Rate in

In [11]:
# change all the numeric data types to float

survey_df_wide_rel_flt = survey_df_wide_rel.iloc[:, 1:22].astype('float64')
survey_df_wide_rel_flt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 21 columns):
Doctors Always Communicated Well                                                     177 non-null float64
Emergency Department Visits, Total                                                   177 non-null float64
Number of Beds                                                                       184 non-null float64
Nurses Always Communicated Well                                                      177 non-null float64
Patients Always Received Help As Soon As They Wanted                                 177 non-null float64
Patients Would Definitely Recommend This Hospital to Friends and Family              177 non-null float64
Percent RN Nursing Staff Hours (Critical Care)                                       180 non-null float64
Percentage of Nursing Hours Worked by Hospital Employed RNs (Critical Care)          176 non-null float64
RN Turnover Rate in the Critical Care Unit     

In [12]:
#convert object type to category

survey_df_wide_rel_cat = pd.DataFrame(survey_df_wide_rel.iloc[:, 0].astype('category'))

survey_df_wide_rel_cat.head()
survey_data = survey_df_wide_rel_cat.join(survey_df_wide_rel_flt)
survey_data.head(2)

survey_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 22 columns):
Hospital                                                                             184 non-null category
Doctors Always Communicated Well                                                     177 non-null float64
Emergency Department Visits, Total                                                   177 non-null float64
Number of Beds                                                                       184 non-null float64
Nurses Always Communicated Well                                                      177 non-null float64
Patients Always Received Help As Soon As They Wanted                                 177 non-null float64
Patients Would Definitely Recommend This Hospital to Friends and Family              177 non-null float64
Percent RN Nursing Staff Hours (Critical Care)                                       180 non-null float64
Percentage of Nursing Hours Worked by Hospital

Now we need to split the hospital names column to pull out just the text descriptions of the hospital names, which will be useful for downstream analyses such as joining data frames and matching up data sources. The unique IDs have already served their purpose and now each row is a unique hospital.

In [15]:
hospitalNames = survey_df_wide_rel["Hospital"].str.split(" ", n=1, expand = True)
survey_data['ID']=hospitalNames[0]
survey_data['Name']=hospitalNames[1]
survey_data.head()

,Hospital,Doctors Always Communicated Well,"Emergency Department Visits, Total",Number of Beds,Nurses Always Communicated Well,Patients Always Received Help As Soon As They Wanted,Patients Would Definitely Recommend This Hospital to Friends and Family,Percent RN Nursing Staff Hours (Critical Care),Percentage of Nursing Hours Worked by Hospital Employed RNs (Critical Care),RN Turnover Rate in the Critical Care Unit,...,Total RN Nursing Hours per Patient Day (Critical Care),Total RN Nursing Hours per Patient Day (Medical-Surgical),Percentage of Nursing Hours Worked by Hospital Employed RNs (Medical-Surgical),Heart Attack 30-Day Mortality Rate,Pneumonia 30-Day Mortality Rate,Percent RN Nursing Staff Hours (Medical-Surgical),"Median Length of Stay for DRG 638, Diabetes with Complications",Median Length of Stay: Septicemia with Multiple Complications,ID,Name
0,hospital_101152: Graham Hospital,85.0,13205.0,87.0,80.0,68.0,60.0,93.18,89.31,0.00,...,16.93,4.76,56.55,NaN,18.0,63.88,2.0,5.0,hospital_101152:,Graham Hospital
1,hospital_101153: Alton Memorial Hospital,76.0,35566.0,181.0,78.0,64.0,71.0,90.26,89.50,7.41,...,15.11,6.90,77.08,12.6,17.2,92.03,3.0,4.0,hospital_101153:,Alton Memorial Hospital
2,hospital_101154: Amita Health Saint Joseph Med...,76.0,57237.0,498.0,77.0,63.0,63.0,90.65,93.52,17.76,...,16.73,6.81,71.54,11.9,16.6,66.85,3.0,5.0,hospital_101154:,Amita Health Saint Joseph Medical Center Joliet
3,hospital_101155: Loyola Gottlieb Memorial Hosp...,78.0,25358.0,247.0,78.0,66.0,63.0,95.07,95.01,3.33,...,15.28,7.12,77.65,12.4,15.7,79.32,2.5,5.0,hospital_101155:,Loyola Gottlieb Memorial Hospital
4,hospital_101156: Northshore Univ HS Evanston H...,81.0,35149.0,354.0,80.0,64.0,77.0,83.26,83.84,13.85,...,16.26,5.75,59.36,9.4,12.8,60.44,3.0,5.0,hospital_101156:,Northshore Univ HS Evanston Hospital


## Now we need to combine the RN turnover columns, taking the maximum turnover rate per hospital, and join the survey data with the employee review data 

Some hospitals have all four medical unit types: Critical Care, Medical-Surgical, Mother/Baby, Neonatal. Very few had Neonatal, so we did not consider that in this analysis. Larger hospitals have all four units, so it would create a lot of missing values if we kept all four unit types in the analysis. Thus, we create a new dataframe and then take the maximum value in any unit within each hospital, to use as our training labels.

In [16]:
turnover = survey_data[["RN Turnover Rate in the Critical Care Unit", 
          "RN Turnover Rate in the Medical-Surgical Unit",
          "RN Turnover Rate in the Mother/Baby Unit",
           "Name"]]
turnover.head()

,RN Turnover Rate in the Critical Care Unit,RN Turnover Rate in the Medical-Surgical Unit,RN Turnover Rate in the Mother/Baby Unit,Name
0,0.00,0.00,0.00,Graham Hospital
1,7.41,7.69,15.38,Alton Memorial Hospital
2,17.76,16.28,7.14,Amita Health Saint Joseph Medical Center Joliet
3,3.33,9.84,0.00,Loyola Gottlieb Memorial Hospital
4,13.85,22.45,5.77,Northshore Univ HS Evanston Hospital


In [20]:
turn2 = pd.DataFrame(turnover[["RN Turnover Rate in the Critical Care Unit", 
          "RN Turnover Rate in the Medical-Surgical Unit",
          "RN Turnover Rate in the Mother/Baby Unit"]].max(axis=1))


survey_data2=survey_data.join(turn2)#.drop(['Hospital', 'RN Turnover', 'Num_Units'], axis=1)
turn2.head(10)

#export the final dataframe to use in our training notebook

survey_data2.to_csv("SurveyData.csv")

,0
0,0.00
1,15.38
2,17.76
3,9.84
4,22.45
5,36.96
6,15.00
7,20.00
8,40.79
9,25.42
